# 6. Делаем модель с помощью класса tf.Module. Пример обучения простой нейросети

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np

2024-10-15 22:48:27.325419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 22:48:27.336369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 22:48:27.339521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#### Один полносвязанный слой нейронной сети

In [2]:
class DenseNN(tf.Module):
    def __init__(self, outputs):
        super().__init__()
        self.outputs = outputs
        self.fl_init=False
    
    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name='w')  
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name='b')
            
            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)
            
            self.fl_init = True
        
        y = x @ self.w + self.b
        return y


In [3]:
model = DenseNN(1)

In [4]:
print(model(tf.constant([[1.0, 2.0]])))
#неверное решение, так как нейронная сеть не обучена

tf.Tensor([[0.08681004]], shape=(1, 1), dtype=float32)


In [5]:
x_train = tf.random.uniform(minval=0, maxval=10, shape=(100, 2))
y_train = [a + b for a, b in x_train]

In [6]:
#функция потерь и оптимизатор для градиентного спуска

loss = lambda x, y: tf.reduce_mean(tf.square(x - y))
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [7]:
epochs = 50

for n in range(epochs):
    for x, y in zip(x_train, y_train):
        x = tf.expand_dims(x, axis=0)
        y = tf.constant(y, shape=(1, 1))
        
        with tf.GradientTape() as tape:
            f_loss = loss(y, model(x))
            
        grads = tape.gradient(f_loss, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))
        
    print(f_loss.numpy())
    
print(model.trainable_variables)

5.734408
0.0021847666
0.10519202
0.10667072
0.09333435
0.08206694
0.07266663
0.0646026
0.057477277
0.051032826
0.045123264
0.03967747
0.0346673
0.03008242
0.025916781
0.022162417
0.018806146
0.015830232
0.0132144205
0.010934967
0.008966438
0.0072823497
0.005855743
0.0046591936
0.003666386
0.0028514625
0.0021898511
0.0016591271
0.0012384442
0.00090945006
0.00065596827
0.00046383493
0.00032087072
0.00021678963
0.00014281429
9.1605485e-05
5.7164638e-05
3.46796e-05
2.045141e-05
1.1715129e-05
6.508003e-06
3.5117637e-06
1.8339051e-06
9.222722e-07
4.469266e-07
2.0606967e-07
9.08185e-08
3.7479367e-08
1.466924e-08
5.2532414e-09
(<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.00015501], dtype=float32)>, <tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[0.99998623],
       [0.9999929 ]], dtype=float32)>)


In [8]:
print(model(tf.constant([[1.0, 2.0]])))

tf.Tensor([[3.000127]], shape=(1, 1), dtype=float32)
